# Network Security — Exercise 5: Domain Name System Security & Network Firewalls

**Noah Link, Jan Pfeifer, Julian Weske**  
**Date:** 20.06.2024

## Domain Name System Security (Time spent: xx h)

### Investigating Legal Websites

Initializing the project and dnsdbq worked just fine:
```
./dnsdbq -u circl -n www.heise.de
;; record times: 2023-10-08 18:53:22 .. 2024-06-12 16:29:03 (~247d 21h 35m)
;; count: 369
193.99.144.85  A  www.heise.de

... skipped ...

;; record times: 2024-01-21 15:44:12 .. 2024-06-11 23:03:38 (~142d 7h 19m)
;; count: 7
www.heise.de  PTR  85.144.99.193.in-addr.arpa
```

### Passive DNS

- **What is passive DNS?**  
  Passive DNS is a method of collecting, storing, and analyzing DNS query and response data from recursive DNS servers. This allows for historical tracking of domain-to-IP mappings without actively querying DNS servers.

- **How can investigations in cybercrime benefit from passive DNS analysis?**
  - Identifying historical domain-IP mappings to uncover the infrastructure used.
  - Tracking changes in domain associations, which can reveal patterns of malicious activity.
  
- **Name two factors that the quality of passive DNS analysis, i.e., the number of returned results, depends on.**
  - The volume and diversity of DNS data sources contributing to the passive DNS database.
  - The time span over which DNS data has been collected and stored.

## IT Security News
Please use passive DNS to analyze the legal website www.heise.de according to the following questions:

In [1]:
import pypdns
auth_details = ('students.informatik.uni-hamburg.de', 'DWW/ymamruvwjRfwo8g8SFaCw1H8zYj5GlxBS8JVWgM=')
client = pypdns.PyPDNS(basic_auth=auth_details)
host = "www.heise.de"


1. How many IPv4 addresses exist for this site? List all addresses (i.e., A records)
you found:

In [2]:
# 1.1.2.1
result = client.rfc_query(host)# , filter_rrtype='A') #todo 
ip_addresses = [record.rdata for record in result]

print("IP addresses for ", host, ": ")
for address in ip_addresses:
    print(address)

IP addresses for  www.heise.de : 
5.8.0.0.2.0.0.0.e.2.7.7.7.7.7.7.1.0.0.1.e.f.3.0.0.e.2.0.2.0.a.2.ip6.arpa
karriere.heise-gruppe.de
monocle.heise.de
www.heisegroup.de
compaliate.heise.de
85.144.99.193.in-addr.arpa
reichweite.heise.de
www.heise.de
www.heise.de
www.heise.de
www.heise.de
www.heise.de
www.telepolis.de
www.heise.de
www.heise.de


2. Is this site using IPv6 as well? If yes, list all addresses (i.e., AAAA records) you
found.

In [3]:
# 1.1.2.2
result = client.rfc_query(host, filter_rrtype='AAAA')
ip_addresses = [record.rdata for record in result]

print("IP addresses for ", host, ": ")
for address in ip_addresses:
    print(address)

IP addresses for  www.heise.de : 
www.heise.de


3. Search for other Fully-Qualified Domain Names (FQDNs) that are also hosted on
the first IPv4 address of www.heise.de.

In [4]:
# 1.1.2.3
results = client.rfc_query(host)

fqdn_set = set()
for result in results:
    if result.rrname:
        fqdn_set.add(result.rrname)

print(f'FQDNs hosted on the same IP address ({host}):')
for fqdn in fqdn_set:
    print(fqdn)

FQDNs hosted on the same IP address (www.heise.de):
www.heise.de
tiktok-developers-site-verification=4hG7PLWNR1XvEcOT4YiLagaIG0dISTp5
tiktok-developers-site-verification=weIoKWwdQgrTvpnCHhaQYbSHVDYzRrPG
tiktok-developers-site-verification=ItHbW0BcRfQvyRg64ke5Hjyr8Kq0tNLw
tiktok-developers-site-verification=XRVQfkPuANzTVYCOWuMGUoo3cuGojxoC
2a02:2e0:3fe:1001:7777:772e:2:85
tiktok-developers-site-verification=FHHUbiCIOtixTBCGm2NCq7EWqXVFDha9
193.99.144.85


4. List all FQDNs that share a common IPv6 address with www.heise.de.

In [5]:
# 1.1.2.4
import socket
result = client.rfc_query(host, filter_rrtype='AAAA')
ipv6_addresses = {record.rdata for record in result}

for ipv6_address in ipv6_addresses:
    query_results = client.rfc_query(ipv6_address, filter_rrtype='AAAA')
    
    if result:
        fqdns = {record.rrname for record in query_results}
        fqdns.discard(host)
        print(f"FQDNs sharing IPv6 address {ipv6_address}:")
        for fqdn in fqdns:
            print(fqdn)
    else:
        print(f"No FQDNs found sharing IPv6 address {ipv6_address}")

    try:
        fqdn = socket.gethostbyaddr(ipv6_address)
        print(f"The FQDN of {ipv6_address} is: {fqdn}")
    except socket.herror as e:
        print(f"Unable to find the FQDN for {ipv6_address}")


FQDNs sharing IPv6 address www.heise.de:
2a02:2e0:3fe:1001:7777:772e:2:85
The FQDN of www.heise.de is: ('www.heise.de', [], ['193.99.144.85'])


### University Website
Please use passive DNS to analyze the legal website www.uni-hamburg.de.
Hint: For this task, you might not be able to find a directly referenced A record, but CNAME records instead that you have to follow. In this case, continue with the most recent alias name

1. How many IPv4 addresses exist for this site? List all addresses (i.e., A records) you found.

In [6]:
# 1.1.3.1
host = "www.uni-hamburg.de"
result = client.rfc_query(host)# , filter_rrtype='A')
ip_addresses = [record.rdata for record in result]

print("IP addresses for ", host, ": ")
for address in ip_addresses:
    print(address)

IP addresses for  www.uni-hamburg.de : 
www.uni-hamburg.de


2. Is this site using IPv6 as well? If yes, list all addresses (i.e., AAAA records) you found.

In [7]:
# 1.1.3.2
result = client.rfc_query(host, filter_rrtype='AAAA')
ip_addresses = [record.rdata for record in result]

print("IP addresses for ", host, ": ")
for address in ip_addresses:
    print(address)

IP addresses for  www.uni-hamburg.de : 



3. Search for other Fully-Qualified Domain Names (FQDNs) that are also hosted on the first IPv4 address of www.uni-hamburg.de.

In [8]:
# 1.1.3.3
import pypdns
auth_details = ('students.informatik.uni-hamburg.de', 'DWW/ymamruvwjRfwo8g8SFaCw1H8zYj5GlxBS8JVWgM=')
client = pypdns.PyPDNS(basic_auth=auth_details)
host = "www.heise.de"
results = client.rfc_query(host)

fqdn_set = set()
for result in results:
    if result.rrname:
        fqdn_set.add(result.rrname)

print(f'FQDNs hosted on the same IP address ({host}):')
for fqdn in fqdn_set:
    print(fqdn)

FQDNs hosted on the same IP address (www.heise.de):
www.heise.de
tiktok-developers-site-verification=4hG7PLWNR1XvEcOT4YiLagaIG0dISTp5
tiktok-developers-site-verification=weIoKWwdQgrTvpnCHhaQYbSHVDYzRrPG
tiktok-developers-site-verification=ItHbW0BcRfQvyRg64ke5Hjyr8Kq0tNLw
tiktok-developers-site-verification=XRVQfkPuANzTVYCOWuMGUoo3cuGojxoC
2a02:2e0:3fe:1001:7777:772e:2:85
tiktok-developers-site-verification=FHHUbiCIOtixTBCGm2NCq7EWqXVFDha9
193.99.144.85



4. List all FQDNs that share a common IPv6 address with www.uni-hamburg.de.

In [9]:
# 1.1.3.4
# Todo sollte eigentlich funktionieren aber ipv6_addresses_uni ist leer
host = "www.uni-hamburg.de"

import socket
result = client.rfc_query(host, filter_rrtype='AAAA')
ipv6_addresses = {record.rdata for record in result}

for ipv6_address in ipv6_addresses:
    query_results = client.rfc_query(ipv6_address, filter_rrtype='AAAA')
    
    if result:
        fqdns = {record.rrname for record in query_results}
        fqdns.discard(host)
        print(f"FQDNs sharing IPv6 address {ipv6_address}:")
        for fqdn in fqdns:
            print(fqdn)
    else:
        print(f"No FQDNs found sharing IPv6 address {ipv6_address}")

    try:
        fqdn = socket.gethostbyaddr(ipv6_address)
        print(f"The FQDN of {ipv6_address} is: {fqdn}")
    except socket.herror as e:
        print(f"Unable to find the FQDN for {ipv6_address}")

5. Find more domain names that do not directly resolve to the IP address of www.uni-hamburg.de but also indirectly via the respective CNAME.

In [14]:
# 1.1.3.5
# todo: check if correct. der kernel crashed hier ab und zu :c
import dns.resolver

try:
    cname_records = dns.resolver.resolve(host, 'CNAME')
    for record in cname_records:
        alias = record.target.to_text().rstrip('.')
        print(f"{host} CNAME -> {alias}")
        while True:
            try:
                cname_records = dns.resolver.resolve(alias, 'CNAME')
                for record in cname_records:
                    alias = record.target.to_text().rstrip('.')
                    print(f"{alias} CNAME -> {alias}")
                aaaa_records = dns.resolver.resolve(alias, 'AAAA')
                if aaaa_records:
                    ipv6_addresses = {record.to_text() for record in aaaa_records}
                    print(f"{alias} AAAA -> {ipv6_addresses}")
                else:
                    print(f"No AAAA records found for {alias}")

                break 
            except dns.resolver.NoAnswer:
                print(f"No CNAME records found for {alias}")
                break  
except dns.resolver.NoAnswer:
    print(f"No CNAME records found for {host}")
except dns.resolver.NXDOMAIN:
    print(f"Domain {host} does not exist")

www.uni-hamburg.de CNAME -> www-fiona.rrz.uni-hamburg.de
No CNAME records found for www-fiona.rrz.uni-hamburg.de


### Investigating Illegal Websites
Please use passive DNS to analyze the illegal streaming website kinox.to.
Hint: This website may not exist anymore, so you can also investigate other illegal websites. You presume that more streaming websites are hosted within the same address block 104.28.21.0/24.

In [10]:
# todo check if correct
'''
def generate_ip_addresses(base_ip, subnet_mask):
    ip_addresses = []
    for i in range(subnet_mask + 1):
        ip = base_ip + '.' + str(i)
        ip_addresses.append(ip)
    return ip_addresses

def find_fqdns_in_subnet(base_ip, subnet_mask):
    ip_addresses = generate_ip_addresses(base_ip, subnet_mask)
    fqdns = {}
    for ip in ip_addresses:
        try:
            host = socket.gethostbyaddr(ip)
            fqdns[ip] = host[0]
        except socket.herror:
            fqdns[ip] = "Unknown host"
        except socket.gaierror as e:
            fqdns[ip] = f"Failed to resolve: {str(e)}"
    return fqdns
'''


'\ndef generate_ip_addresses(base_ip, subnet_mask):\n    ip_addresses = []\n    for i in range(subnet_mask + 1):\n        ip = base_ip + \'.\' + str(i)\n        ip_addresses.append(ip)\n    return ip_addresses\n\ndef find_fqdns_in_subnet(base_ip, subnet_mask):\n    ip_addresses = generate_ip_addresses(base_ip, subnet_mask)\n    fqdns = {}\n    for ip in ip_addresses:\n        try:\n            host = socket.gethostbyaddr(ip)\n            fqdns[ip] = host[0]\n        except socket.herror:\n            fqdns[ip] = "Unknown host"\n        except socket.gaierror as e:\n            fqdns[ip] = f"Failed to resolve: {str(e)}"\n    return fqdns\n'

1. Search for FQDNs that are hosted within the address block 104.28.21.0/24. Describe your working steps.

In [11]:
# 1.2.1
# todo
base_ip = '104.28.21'
subnet_mask = 255

# fqdns = find_fqdns_in_subnet(base_ip, subnet_mask)

# for ip, fqdn in fqdns.items():
#      print(f"IP: {ip} resolves to FQDN: {fqdn}")


2. Come up with at least four more appropriate search strings for filtering the FQDNs.

In [ ]:
# todo

3. Apply your search strings to the full list of FQDNs and list the filtered names.

In [ ]:
# todo

## DNS and Firewall Evasion (Time spent: xx h)



### DNS Mechanisms and Evasion Techniques

1. Why is DNS often used to bypass firewalls, and why is this a popular attack vector?

DNS is can be used to bypass firewalls because it operates over port 53, which is often allowed through firewalls for essential network functionality. This makes DNS traffic a convenient channel for attackers to tunnel malicious data, evade traditional firewall rules. The difficulty in inspecting DNS traffic thoroughly without specialized tools also contributes to its popularity as an attack vector. Robust DNS monitoring and filtering solutions are necessary to mitigate these risks.

2. Explain the process of how a DNS tunnel works from the client request through to the response.

A DNS tunnel works by encoding data into DNS queries sent by a client to a DNS server. The server extracts this data from the queries and sends back responses that contain the encoded information or acknowledgments. The client then decodes the data from the server's responses, enabling covert communication through DNS channels, therby bypassing security measures.

3. Describe a method to further cloak traffic via DNS tunneling. Provide a detailed description and analyze the overhead involved with concrete numbers and percentages.

    1. Payload Encryption: Encrypt data using AES-256 before embedding it into DNS queries to prevent interception. An attacker should be able to exchange keys in advance. This would add computational effort.
    2. Randomize query structure, timing, and data chunk order to evade pattern detection. Introduce random delays of 50-200 milliseconds between queries. This increases latency and network utilization with random delays.


### DoH, DoT Implementation and Analysis

1. Implement a subset of DNS over HTTPS (DoH) and DNS over TLS (DoT) to query an A record. Demonstrate the implementation by querying a public DNS server.

1.1 DNS over HTTPS Implementation:

In [12]:
import requests

def query_doh(domain):
    doh_url = "https://cloudflare-dns.com/dns-query"
    headers = {"Accept": "application/dns-json"}
    params = {"name": domain, "type": "A"}

    try:
        response = requests.get(doh_url, headers=headers, params=params)
        response.raise_for_status()
        answer = response.json()
        if "Answer" in answer:
            return answer["Answer"]
        else:
            print("No 'Answer' field in the response.")
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

domain = "example.com"
result = query_doh(domain)
if result:
    for record in result:
        print(f"{record['name']} has A record {record['data']}")
else:
    print("No A record found.")


example.com has A record 93.184.215.14


1.2 DNS over TLS Implementation:

Cant get it to run:
https://github.com/melvilgit/DNS-OVER-TLS/blob/master/test.py
https://community.cloudflare.com/t/python-ssl-for-connecting-with-cloudflare-1-1-1-1-853/33561

In [13]:
# Todo does not work :c BufferError 

import socket
import ssl
from dnslib import DNSRecord, A

SERVER_HOST = '1.1.1.1'
SERVER_PORT = 853

domain = 'example.com'
query = DNSRecord.question(domain, 'A')
query_bytes = query.pack()
message_length = len(query_bytes).to_bytes(2, byteorder='big')
query_with_length = message_length + query_bytes
context = ssl.create_default_context()

with socket.create_connection((SERVER_HOST, SERVER_PORT)) as sock:
    with context.wrap_socket(sock, server_hostname=SERVER_HOST) as ssock:
        ssock.send(query_with_length)
        response_bytes = b''
        while True:
            chunk = ssock.recv(4096)
            if not chunk:
                break
            response_bytes += chunk

'''A
response = DNSRecord.parse(response_bytes)
answers = response.rr
if answers:
    for rr in answers:
        if rr.rtype == A:
            print(f"{domain} -> {rr.rdata}")
else:
    print(f"No answers found for {domain}")
'''

'A\nresponse = DNSRecord.parse(response_bytes)\nanswers = response.rr\nif answers:\n    for rr in answers:\n        if rr.rtype == A:\n            print(f"{domain} -> {rr.rdata}")\nelse:\n    print(f"No answers found for {domain}")\n'

2. What are the drawbacks of DoH and DoT, and how could these drawbacks be addressed?

DoH and DoT could face firewall blocking. Additionally, encrypting DNS queries and responses can lead to higher latency. Moreover, adopting DoH might route traffic through a few major providers, potentially restricting DNS options.

To address these concerns, establishing standards and integrating DoH and DoT into current networks could provide solutions. Furthermore, encouraging diverse companies to offer DoH and DoT services would help prevent DNS centralization.